<h1 style="text-align: center">Real Time Farmer Counting for Specific Zones</h1>

In [ ]:
from ultralytics import YOLO
import cv2
import cvzone
import math
from sort import *

In [ ]:
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

In [ ]:
model = YOLO("weights/yolov8l.pt")

In [ ]:
cap = cv2.VideoCapture("video/Farmers.mp4")


In [ ]:
mask = cv2.imread("masks/Field Mask.mp4")

In [ ]:
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)
unique_ids = set()

while True:
    success, img = cap.read()
    # imgRegion = cv2.bitwise_and(img, mask)
    results = model(img, stream=True)
    detections = []

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            w, h = x2 - x1, y2 - y1
            cvzone.cornerRect(img, (x1, y1, w, h))
            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            currentArray = [x1, y1, x2, y2, conf]
            detections.append(currentArray)

    resultsTracker = tracker.update(np.array(detections))

    for x1, y1, x2, y2, id in resultsTracker:
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        w, h = x2 - x1, y2 - y1
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
        cvzone.putTextRect(img, f'Farmer ID:{int(id)}', (max(0, x1), max(35, y1)),
                           scale=2, thickness=3, offset=10)

        unique_ids.add(id)

    cv2.putText(img, f"Total Farmers: {len(unique_ids)}", (50, 50), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)

    cv2.imshow("Real Time Fruit Counting", img)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 320x640 7 persons, 1 horse, 1286.4ms
Speed: 0.0ms preprocess, 1286.4ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 persons, 1 horse, 1274.8ms
Speed: 2.0ms preprocess, 1274.8ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 persons, 1 horse, 1183.7ms
Speed: 0.0ms preprocess, 1183.7ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 persons, 1 horse, 1408.0ms
Speed: 0.0ms preprocess, 1408.0ms inference, 14.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 persons, 1 horse, 1336.2ms
Speed: 12.4ms preprocess, 1336.2ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
